In [220]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

import os

#For inline plotting 
%matplotlib inline                 
%config InlineBackend.figure_format = 'svg'

# set plot properties to seaborn globally
plt.style.use("seaborn-v0_8-white")

In [221]:
## importing the description of the attributes as a dictionary

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

xls = pd.ExcelFile('LCDataDictionary.xlsx')
#print(xls.sheet_names)  #to see available sheets

# descriptions of the data columns are in the 'LoanStats' sheet
loan_data_desc = xls.parse('LoanStats')

# drop unnecessary rows that does not have a description in the description df
loan_data_desc.dropna(inplace = True)

# replace a weird looking one: total_rev_hi_lim \xa0
loan_data_desc.LoanStatNew = loan_data_desc.LoanStatNew.str.replace(r'\s+', '', regex=True)

# turn into a simple dictionary for easy access to column descriptions
col_desc_dict = loan_data_desc.set_index('LoanStatNew')['Description'].to_dict()

-----
### **1. Probability of Default (PD) Model and its Validation**
------

#### **1.1 PD Model: Logistic Regression**
------

In [222]:

loan_Xt = pd.read_csv('PP_FE_data/loan_data_train_pp.csv', index_col = 0)
loan_Xval = pd.read_csv('PP_FE_data/loan_data_val_pp.csv', index_col = 0)
loan_yt = pd.read_csv('PP_FE_data/loan_data_target.csv', index_col = 0)
loan_yval = pd.read_csv('PP_FE_data/loan_data_target_val.csv', index_col = 0)

reference_bins = pd.read_csv('PP_FE_data/reference_bins.csv', index_col = 0)

In [223]:
loan_Xt.head()

,grade:A,grade:B,grade:C,grade:D,grade:G-F-E,sub_grade:A4-A3-A2-A1,sub_grade:B1-A5,sub_grade:B2,sub_grade:B3,sub_grade:B4,...,revol_util:47.05-57.58,revol_util:57.58-77.35,revol_util:77.35-89.75,revol_util:>89.75,annual_inc:<37038.50,annual_inc:37038.50-50017.00,annual_inc:50017.00-66097.50,annual_inc:66097.50-80047.00,annual_inc:80047.00-111112.30,annual_inc:>111112.30
0,True,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,True,False,False
1,False,False,True,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
2,True,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
3,False,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


##### Dropping the reference categories for all the variables in the training and validation set
----

To avoid the dummy trap, we will drop the reference category from our training and validation sets. We will use the reference category as a benchmark! 

In [224]:
reference_bins

,Opt_Ref_Bin
grade,['G' 'F' 'E']
sub_grade,['F5' 'G1' 'G3' 'G5' 'F4' 'G4' 'F3' 'G2' 'F2' 'E5' 'F1' 'E4' 'E2' 'E3']
home_ownership,['OTHER' 'NONE' 'RENT']
verification_status,['Verified']
purpose,['small_business' 'educational' 'moving' 'house' 'other'\n 'renewable_energy' 'medical']
initial_list_status,['f']
inq_last_6mths,"[2.50, inf)"
term_in_months,"[48.00, inf)"
int_rate,"[20.23, inf)"
dti,"[26.44, inf)"


In [225]:
reference_cols = ['grade:G-F-E', 'sub_grade:F5-G1-G3-G5-F4-G4-F3-G2-F2-E5-F1-E4-E2-E3', 'home_ownership:OTHER-NONE-RENT', 'verification_status:Verified'
                  ,'purpose:small_business-educational-moving-house-other-renewable_energy-medical', 'initial_list_status:f', 'inq_last_6mths:3-33', "term_in_months:60"
                  ,'int_rate:>20.23', 'dti:>26.44', 'revol_util:>89.75', 'tot_cur_bal:<81646.00', 'annual_inc:<37038.50']

In [226]:
loan_Xt = loan_Xt.drop(reference_cols, axis = 1)
loan_Xval = loan_Xval.drop(reference_cols, axis = 1)

In [227]:
correlation_matrix = loan_Xt.corr(method='pearson')

# Keep only lower triangle (excluding diagonal)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=0)
lower_triangle = correlation_matrix.mask(mask)

# Filter pairs with high correlation (e.g., > 0.8)
high_corr_pairs = lower_triangle.where(lower_triangle.abs() > 0.5).stack().reset_index()
high_corr_pairs.columns = ['var_1', 'var_2', 'corr']

In [228]:
high_corr_pairs

,var_1,var_2,corr
0,sub_grade:A4-A3-A2-A1,grade:A,0.819808
1,sub_grade:D3-D2,grade:D,0.610644
2,purpose:wedding-vacation-debt_consolidation,purpose:car-credit_card,-0.677186
3,inq_last_6mths:1,inq_last_6mths:0,-0.645266
4,int_rate:<7.50,grade:A,0.591942
5,int_rate:<7.50,sub_grade:A4-A3-A2-A1,0.713652
6,int_rate:7.50-9.50,grade:A,0.663474
7,int_rate:7.50-9.50,sub_grade:B1-A5,0.532926
8,int_rate:9.50-12.01,grade:B,0.687340
9,int_rate:9.50-12.01,sub_grade:B2,0.503476


`grade` and `sub_grade` are essentially the same, so as expected we see high correlations between these variables, to avoid collinearity, we will drop `sub_grade` which will keep our model simple. On the other hand, `int_rate` is also highly correlated with `grade`. This makes sense as the former describes the interest rate applied to the loan which strongly depends on the loan grade `grade` assigned by the lending club:

In [229]:
col_desc_dict['int_rate']

'Interest Rate on the loan'

In [230]:
col_desc_dict['grade']

'LC assigned loan grade'

- if we drop `sub_grade` we thus need to drop `grade` as well to avoid multi-collinearity. Recall that `int_rate` was the variable that has the highest `IV` in the dataset!, so it makes sense to keep it and drop `grade` together with `sub_grade` as well.  

In [231]:
sub_grade_col_list = ['sub_grade:A4-A3-A2-A1', 'sub_grade:B1-A5', 'sub_grade:B2', 'sub_grade:B3', 'sub_grade:B4', 'sub_grade:B5', 'sub_grade:C1', 
                      'sub_grade:C2', 'sub_grade:C3','sub_grade:C4', 'sub_grade:D1-C5', 'sub_grade:D3-D2','sub_grade:E1-D5-D4']

grade_col_list = ["grade:A", "grade:B"	,"grade:C",	'grade:D']

In [232]:
loan_Xt = loan_Xt.drop(grade_col_list, axis = 1)
loan_Xval = loan_Xval.drop(grade_col_list, axis = 1)

loan_Xt = loan_Xt.drop(sub_grade_col_list, axis = 1)
loan_Xval = loan_Xval.drop(sub_grade_col_list, axis = 1)

In [233]:
loan_Xt.head()

,home_ownership:MORTGAGE-ANY,home_ownership:OWN,verification_status:Not-Verified,verification_status:Source-Verified,purpose:car-credit_card,purpose:home_improvement-major_purchase,purpose:wedding-vacation-debt_consolidation,initial_list_status:w,term_in_months:36,inq_last_6mths:0,...,revol_util:24.45-38.78,revol_util:38.78-47.05,revol_util:47.05-57.58,revol_util:57.58-77.35,revol_util:77.35-89.75,annual_inc:37038.50-50017.00,annual_inc:50017.00-66097.50,annual_inc:66097.50-80047.00,annual_inc:80047.00-111112.30,annual_inc:>111112.30
0,True,False,False,True,False,False,True,False,True,False,...,False,False,True,False,False,False,False,True,False,False
1,True,False,True,False,False,False,True,False,True,False,...,False,False,True,False,False,False,False,False,False,True
2,True,False,False,False,False,False,True,False,True,False,...,False,False,False,False,True,False,False,False,True,False
3,False,False,False,True,False,False,True,False,True,False,...,False,False,False,True,False,True,False,False,False,False
4,True,False,False,False,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False


##### Model Estimation
------

In [308]:
log_reg = LogisticRegression(C = 0.05, class_weight={0: 5, 1: 1})

log_reg.fit(loan_Xt, loan_yt.to_numpy().ravel())

LogisticRegression(C=0.05, class_weight={0: 5, 1: 1})

In [309]:
# Accuracy score of the model 

log_reg.score(loan_Xt, loan_yt.to_numpy().ravel())

0.7879730207920049

Accuracy score = (`TP + TN`) / `total_obs`

In [310]:
# model parameters 

feature_names = np.insert(loan_Xt.columns.values, 0, 'intercept')

model_params_dfs = {}

model_params_dfs['M1'] = pd.DataFrame(columns = ["feat_name"], data = feature_names)

model_params_dfs['M1'].loc[:,"reg_coef"] = np.insert(log_reg.coef_.T, 0, log_reg.intercept_.T)


In [311]:
model_params_dfs['M1']

,feat_name,reg_coef
0,intercept,-1.564556
1,home_ownership:MORTGAGE-ANY,0.050708
2,home_ownership:OWN,0.127162
3,verification_status:Not-Verified,0.102445
4,verification_status:Source-Verified,0.112740
5,purpose:car-credit_card,0.320360
6,purpose:home_improvement-major_purchase,0.199725
7,purpose:wedding-vacation-debt_consolidation,0.204422
8,initial_list_status:w,0.292091
9,term_in_months:36,0.119450


To assess the statistical significance of the model parameters, we can use `statsmodels` because unfortunately, `sklearn` does not have a module that can check `multi-variate` `p-values` of the model parameters. 

In [203]:
class_counts = loan_yt.value_counts()
weight_for_0 = 1 / class_counts[0]
weight_for_1 = 1 / class_counts[1]

In [179]:
# Assign weights to each row
weights = loan_yt.squeeze().apply(lambda v: weight_for_1 if v == 1. else weight_for_0)

In [183]:
import statsmodels.api as sm

# Add a constant (intercept term)
X_with_const = sm.add_constant(loan_Xt).astype(float)

# Fit logistic regression model
#logit_model = sm.Logit(loan_yt.to_numpy().ravel(), X_with_const, weights = weights)
#result = logit_model.fit(maxiter = 200)

# Get summary with p-values
#print(result.summary())

# Use GLM with Binomial family and weights
glm_model = sm.GLM(loan_yt.to_numpy().ravel(), X_with_const, 
                   family=sm.families.Binomial(), 
                   freq_weights=weights)

result = glm_model.fit()

print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:               373028
Model:                            GLM   Df Residuals:                      -37
Model Family:                Binomial   Df Model:                           38
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1.2813
Date:                Thu, 01 May 2025   Deviance:                       2.5627
Time:                        17:12:49   Pearson chi2:                     2.00
No. Iterations:                     4   Pseudo R-squ. (CS):          5.628e-07
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

Modifying the `.fit()` method of `LogisticRegression()` to take into account multi-variate p-values

In [291]:
from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:
    
    def __init__(self,*args,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)

    def fit(self,X,y):
        # Add intercept term manually
         
        X_intercept = np.hstack([np.ones((X.shape[0], 1)), X])
        
        self.model.fit(X, y)
        
        # Decision function using X with intercept
        decision = self.model.decision_function(X)
        
        # Compute Fisher Information Matrix
        denom = 2.0 * (1.0 + np.cosh(decision))
        
        denom = np.tile(denom, (X_intercept.shape[1], 1)).T
        F_ij = np.dot((X_intercept / denom).T, X_intercept)
        
        # Inverse of Fisher Info = Covariance matrix of parameters
        Cramer_Rao = np.linalg.inv(F_ij)
        sigma_estimates = np.sqrt(np.diag(Cramer_Rao))
        
        # Combine intercept and coefficients
        full_coef = np.hstack([self.model.intercept_, self.model.coef_.ravel()])
        
        # Z-scores and p-values
        z_scores = full_coef / sigma_estimates
        p_values = [stat.norm.sf(abs(z)) * 2 for z in z_scores]
        
        # Store results
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values
        #self.model.fit(X,y)
        #denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        #denom = np.tile(denom,(X.shape[1],1)).T
        #F_ij = np.dot((X / denom).T,X)
        #Cramer_Rao = np.linalg.inv(F_ij)
        #sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        #z_scores = self.model.coef_[0] / sigma_estimates
        #p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores]
        #self.coef_ = self.model.coef_
        #self.intercept_ = self.model.intercept_
        #self.p_values = p_values

In [312]:


log_reg2 = LogisticRegression_with_p_values(C = 0.05, class_weight={0: 5, 1: 1})

log_reg2.fit(loan_Xt, loan_yt.to_numpy().ravel())


In [313]:
model_params_dfs['M1_mod'] = pd.DataFrame(columns = ["feat_name"], data = feature_names)

model_params_dfs['M1_mod'].loc[:,"reg_coef"] = np.insert(log_reg2.coef_.T, 0, log_reg2.intercept_.T)

p_values = log_reg2.p_values # the class does not calculate p-val for the intercept

model_params_dfs['M1_mod']['p_value'] = p_values

In [314]:
# of course the regression coefficients are same as before!

model_params_dfs['M1_mod']

,feat_name,reg_coef,p_value
0,intercept,-1.564556,0.000000e+00
1,home_ownership:MORTGAGE-ANY,0.050708,7.158888e-08
2,home_ownership:OWN,0.127162,1.714661e-21
3,verification_status:Not-Verified,0.102445,9.134390e-28
4,verification_status:Source-Verified,0.112740,7.538088e-38
5,purpose:car-credit_card,0.320360,3.389245e-109
6,purpose:home_improvement-major_purchase,0.199725,2.184409e-29
7,purpose:wedding-vacation-debt_consolidation,0.204422,2.099977e-57
8,initial_list_status:w,0.292091,5.873228e-306
9,term_in_months:36,0.119450,2.964108e-37


In [315]:
# add the results from statsmodel to the same df

model_params_dfs['M1_mod']['reg_coef:SM'] = result.params.values
model_params_dfs['M1_mod']['p_val:SM'] = result.pvalues.values

model_params_dfs['M1_mod']

,feat_name,reg_coef,p_value,reg_coef:SM,p_val:SM
0,intercept,-1.564556,0.000000e+00,-2.076672,0.848974
1,home_ownership:MORTGAGE-ANY,0.050708,7.158888e-08,0.046983,0.990346
2,home_ownership:OWN,0.127162,1.714661e-21,0.126044,0.981845
3,verification_status:Not-Verified,0.102445,9.134390e-28,0.103227,0.978948
4,verification_status:Source-Verified,0.112740,7.538088e-38,0.111970,0.975379
5,purpose:car-credit_card,0.320360,3.389245e-109,0.324311,0.956130
6,purpose:home_improvement-major_purchase,0.199725,2.184409e-29,0.205132,0.977583
7,purpose:wedding-vacation-debt_consolidation,0.204422,2.099977e-57,0.209797,0.967725
8,initial_list_status:w,0.292091,5.873228e-306,0.293026,0.928629
9,term_in_months:36,0.119450,2.964108e-37,0.107432,0.977700


The fit coefficients differ at times slightly, however there is mainly an agreement between `LogisticRegression()` and `statsmodels`. 

Next we can decide which coefficients are statistically significant. If all the dummy categories of a variable are statistically significant we keep them if all not statistical significant we can remove all of them. On the other hand, if some categories of a variable is not statistically significant, we should still keep all the categories, as removing them would not make sense! 

According to this logic, for our case all the dummy variables we choose seems to be statistically significant, as can be seen from the table above!

**PD Model**: With a PD model we aim to assess the probability that a borrower defaults on his/her loan. For the data we are working with we assigned the label $1$ for good borrowers and $0$ for bad borrowers. For a given set of features PD is mathematical expressed via the logistic function or the sigmoid function: 

$$
p(Y = 1 \,|\, \vec{X}) = \frac{\mathrm{e}^{\beta_0 + \beta_1 X_1 + \dots \beta_p X_p}}{1 + \mathrm{e}^{\beta_0 + \beta_1 X_1 + \dots \beta_p X_p}}
$$

so that the probability of the second outcome read as 
$$
p(Y = 0 \,|\, \vec{X}) = 1 - p(Y = 1 | \vec{X}) = \frac{1}{1 + \mathrm{e}^{\beta_0 + \beta_1 X_1 + \dots \beta_p X_p}}
$$

So that the odds are given by 

$$
\textrm{odds} = \frac{p(Y = 1 \,|\, \vec{X})}{p(Y = 0 \,|\, \vec{X})} = \mathrm{e}^{\beta_0 + \beta_1 X_1 + \dots \beta_p X_p}
$$

This tells us that a given observation's odd of being a good credit increases for positive coefficients $\beta_i$'s. Notice that the log-odds is equivalent to a linear regression model: 

$$
\ln \textrm{odds} = \beta_0 + \beta_1 X_1 + \dots \beta_p X_p. 
$$

For the purpose of interpretability, we are working with features that are all dummy variables that takes values of 0 or 1. So the difference of the logarithms of odds when a predictor takes value 1 and 0 gives us the coefficient corresponding to that variable:

$$
\ln \textrm{odds}(X_j = 1)  - \ln \textrm{odds}(X_j = 0) = \ln \left(\frac{\textrm{odds}(X_j = 1)}{\textrm{odds}(X_j = 0)}\right) = \beta_j
$$



This expression allow us to determine relative odds of non-default when a predictor is "turned on" with respect to the case when it is turned off. The latter in fact corresponds to the reference categories that we choose for all variables we work with. Therefore, the coefficients $\beta_j$'s are relative to a benchmark!

For example odds of someone who owns a house being a good borrower with respect to someone rents a house is given by:  

In [213]:
np.exp(0.130343)

1.1392190684587724

Similarly, we can find relations between different categories of the SAME variable! 

$$
\frac{\textrm{odds}(X_j = 1)}{\textrm{odds}(X_{j'} = 1)} =\frac{\mathrm{e}^{\beta_j}\, \textrm{odds}(X_j = 0)}{\mathrm{e}^{\beta_{j'}}\, \textrm{odds}(X_j = 0)} = \frac{\mathrm{e}^{\beta_j}}{\mathrm{e}^{\beta_{j'}}} = \mathrm{e}^{\beta_j - \beta_{j'}}
$$

So odds of someone who owns a house to be a good borrower with respect to someone that with mortgage is:

In [214]:
np.exp(0.130343 - 0.053951)

1.0793856103960393

It is important to emphasize that such comparisons are possible only between categories originating from the same variable/predictor!

#### Model Validation
-----

In [316]:
# predictions for the target class on the validation set
# makes predictions for a given pd depending on if pd > or < than the defualt cut-off = 0.5

yval_hat = log_reg2.model.predict(loan_Xval)

In [317]:
# to adjust the model, it is therefore more important to get the probability predictions for the target class 
# returns an array of n x 2 where the first column is default (0) and (1) non-default 
yval_proba = log_reg2.model.predict_proba(loan_Xval)[:,1]

yval_proba

array([0.59314666, 0.52074011, 0.6833757 , ..., 0.8716901 , 0.81252203,
       0.8262049 ])

In [318]:
val_targets_df = loan_yval.copy()

val_targets_df.columns = ['true']

val_targets_df['pred_prob'] = yval_proba

In [319]:
val_targets_df.head()

,true,pred_prob
0,1,0.593147
1,1,0.520740
2,1,0.683376
3,1,0.685030
4,1,0.836782


In [320]:
# confusion_matrix with the default cut-off 0.5

prob_cut_off = 0.5

val_targets_df['predicted_0p5'] = np.where(val_targets_df['pred_prob'] >= prob_cut_off, 1, 0)


In [321]:
val_targets_df.head()

,true,pred_prob,predicted_0p5
0,1,0.593147,1
1,1,0.520740,1
2,1,0.683376,1
3,1,0.685030,1
4,1,0.836782,1


In [322]:
# confusion matrix 

conf_matrix = pd.crosstab(val_targets_df['true'].values,val_targets_df['predicted_0p5'].values, rownames= ['Actual'], colnames = ['Predicted'])

conf_matrix


Predicted,0,1
Actual,,
0,3534,6656
1,13013,70054


Only increasing the probability threshold for good/bad loans we can get meaningful results. If threshold is 0.5, to model v